In [1]:
# USAGE
# Start the server:
# 	python run_server.py
# Submit a request via cURL:
# 	curl -X POST -F image=@dog.jpg 'http://localhost:5000/predict'
# Submita a request via Python:
#	python simple_request.py

# import the necessary packages
import dill
import pandas as pd
import os
dill._dill._reverse_typemap['ClassType'] = type
#import cloudpickle
import flask
import logging
from logging.handlers import RotatingFileHandler

# initialize our Flask application and the model
app = flask.Flask(__name__)
model = None

handler = RotatingFileHandler(filename='app.log', maxBytes=100000, backupCount=10)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.addHandler(handler)

def load_model(model_path):
	# load the pre-trained model
	global model
	with open(model_path, 'rb') as f:
		model = dill.load(f)

@app.route("/", methods=["GET"])
def general():
	return "Welcome to fraudelent prediction process"

@app.route("/predict", methods=["POST"])
def predict():
	# initialize the data dictionary that will be returned from the
	# view
	data = {"success": False}

	# ensure an image was properly uploaded to our endpoint
	if flask.request.method == "POST":
		title, text = "", "qq"
		request_json = flask.request.get_json()
		if request_json["title"]:
			title = request_json['title']
		if request_json["text"]:
			text = request_json['text']

		print(title)

		try:
			preds = model.predict_proba(pd.DataFrame({"title": [title],
												  "text": [text]
												  }))
		except AttributeError as e:
			logger.warning(f'{title} Exception: {str(e)}')
			data['predictions'] = str(e)
			data['success'] = False
			return flask.jsonify(data)
		data["predictions"] = preds[:, 1][0]
		data["title"] = title
		# indicate that the request was a success
		data["success"] = True
		print('Все прошло хорошо')

	# return the data dictionary as a JSON response
	return flask.jsonify(data)

# if this is the main thread of execution first load the model and
# then start the server
if __name__ == "__main__":
	print(("* Loading the model and Flask starting server..."
		"please wait until server has fully started"))
	modelpath = "/Users/4а/MLbusiness/course_project/logreg_pipeline.dill"
	load_model(modelpath)
	app.run()

* Loading the model and Flask starting server...please wait until server has fully started
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -


NewSTAR
Все прошло хорошо
American Dream, Revisited
Все прошло хорошо
Clintons Are Under Multiple FBI Investigations as Agents Are Stymied
Все прошло хорошо
The FBI Can’t Actually Investigate a Candidate Such as Hillary Clinton.
Все прошло хорошо
Confirmed: Public overwhelmingly (10-to-1) says media want Hillary to win


127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:28] "POST /pred

Все прошло хорошо
Nanny In Jail After Force Feeding Baby To Death
Все прошло хорошо
Media Roll Out Welcome Mat for ‘Humanitarian’ War in Syria
Все прошло хорошо
Hillary Clinton accepts nomination with 'boundless confidence in America's promise'
Все прошло хорошо
Police Turn In Badges Rather Than Incite Violence Against Standing Rock Protesters: Report
Все прошло хорошо
South Carolina police officer charged with murder after shooting man during traffic stop
Все прошло хорошо
Tony Blair helpfully describes Remain voters as ‘insurgents’
Все прошло хорошо
BREAKING: Trump VP Plane Slides Off Runway at LaGuardia Airport in NYC!
Все прошло хорошо
Clinton, under fire for oil and gas donations, once hit Obama for same reason
Все прошло хорошо
Can Paul Ryan and Donald Trump coexist within the Republican Party?
Все прошло хорошо
Europe’s Morality Crisis: Euthanizing the Mentally Ill
Все прошло хорошо
Project Veritas 4: Robert Creamer's Illegal $20,000 Foreign Wire Transfer Caught On Tape
Все прош

...

Hillary Clinton in HUGE Trouble After America Noticed SICK Thing Hidden in this Picture... * LIBERTY WRITERS NEWS
Все прошло хорошо
Report Says Trade Deal Would Boost U.S. Economy, But Opponents Say No
Все прошло хорошо
Trump’s Gettysburg Address against the New World Order
Все прошло хорошо
With Iran deal, Obama makes bad history
Все прошло хорошо
BREAKING: Irrefutable Proof Obama Lied to Protect Hillary Clinton’s Run for the White House Read
Все прошло хорошо
Rubio’s parting shot at Trump
Все прошло хорошо
The Conspiracy “Theory” Conspiracy [Video Documentary]
Все прошло хорошо
First Iraqi Troops Enter Mosul, But Fighting Remains Largely in Suburbs
Все прошло хорошо
How Clinton plans to stop Sanders' momentum
Все прошло хорошо
Kerry: World leaders take step to end Syria war and spreading terror concerns, but disagreements persist
Все прошло хорошо
Crucial Rule Is Delayed a Year for Obama’s Health Law
Все прошло хорошо
Black and a member of the 1% elite
Все прошло хорошо
Clinton Goes 

127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /pred

Many Popular Tea Bags Contain Alarming Amounts of Deadly Pesticides (avoid these brands like the plague)
Все прошло хорошо
Clinton, Sanders Let Loose in New York Debate | RealClearPolitics
Все прошло хорошо
More Reports Of Votes Flipping From Trump To Clinton In Texas ….election Officials Dismiss Concerns… An Ongoing Cbs4 Voter Fraud Investigation Finds People Voting Twice
Все прошло хорошо
Chicago court to hear 'natural-born' case to knock Ted Cruz off ballot
Все прошло хорошо
Washington's Embargo on Cuba Not Effective, Isolated US - White House
Все прошло хорошо
Chinese political dissidents are having their organs cut from their bodies for 'transplant tourists'
Все прошло хорошо
“We are losing control of the streets,” say police as Angela Merkel’s Germany descends into chaos and lawlessness
Все прошло хорошо
Is the GOP's stop Trump campaign too late?
Все прошло хорошо
Robert Vaughn dies after hearing Trumphole wins
Все прошло хорошо
US/Russian Relations Sour Further As Putin Refuses 

127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [11/Feb/2022 10:53:34] "POST /predict HTTP/1.1" 200 -



Все прошло хорошо
She Drank Carrot Juice Every Day for 8 Months and Got Rid of CANCER!
Все прошло хорошо
White House Monitoring Iran Aid Ship to Yemen
Все прошло хорошо
The Oregon standoff and America’s double standards on race and religion
Все прошло хорошо
New O'Keefe Video: Clinton Campaign, DNC Coordinated With Organizations To Incite Violence At Trump Events
Все прошло хорошо
